In [1]:

import os
os.environ["OMP_NUM_THREADS"] = "32"
from graph_tool.all import *
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.covariance import LedoitWolf, OAS
import matplotlib.pyplot as plt
import matplotlib.cm as mpl
import seaborn as sns
import statsmodels.api as sm
from multipy.fdr import qvalue
from multipy.fdr import lsu

import dill

In [2]:
# Loading blocks...
def load_blocks(blocks):
    with open (blocks, "rb") as fh:
        bs = dill.load(fh)[0:6]
    return bs

In [ ]:
g_path = '../snakemake-layer/cache/trimmed_graph/fdr-1e-3/layered/'
tissues = ['head', 'body']
graphs = {f'{tissue}':load_graph(g_path + f'{tissue}.xml.gz') for tissue in tissues}
b_path = '../snakemake-layer/cache/MCMC/blocks/fdr-1e-3/layered/'
blocks = {f'{tissue}':load_blocks(b_path + f'{tissue}.dill') for tissue in tissues}

In [9]:
state = {}
for l in tissues:
    g = graphs[l]
    bs = blocks[l]
    # Arctan transform on correlations...
    corr = g.edge_properties["spearman"]
    g.ep.weight = g.new_edge_property("double", (2*np.arctanh(corr.a)))

    # Creating nested block model...
    state[l] = minimize_nested_blockmodel_dl(g, init_bs=bs,
                                             state_args=dict(base_type=LayeredBlockState,
                                                             ec=g.ep.layer,layers=True,
                                                             recs=[g.ep.weight],
                                                             rec_types=["real-normal"]))


In [ ]:
for l in tissues:
    s = state[l]
    g = graphs[l]
    g.vp.level_0 = g.new_vertex_property("double", s.get_bs()[0])
    s.draw( eorder=g.ep.weight,
            #edge_pen_width = gf_body.ep.weight,
            edge_color=prop_to_size(g.ep.weight, mi=-4, ma=4, power=1, log=False),
            ecmap=(mpl.inferno, .6), 
            edge_gradient=[], 
            vertex_size = 4,
            hvertex_size = 25,
            hedge_pen_width = 3,
            vertex_color = g.vp.level_0,
            vertex_fill_color = g.vp.level_0,
            subsample_edges = 30000,
            output = l + ".png", 
            output_size=(2000, 2000))